In [1]:
%matplotlib widget

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.pipeline import Pipeline

In [2]:
df = pd.read_csv('data/all_timeline_with_features.csv')

In [3]:
df.columns

Index(['date', 'countrycode', 'countrylabel', 'totalcases', 'totaldeaths',
       'totalrecovered', 'ft01_previous_day_totalcases',
       'ft02_previous_day_totaldeaths', 'ft03_previous_day_totalrecovered',
       'ft04_new_cases_per_day', 'ft05_new_deaths_per_day',
       'ft06_new_recovered_per_day', 'ft07_previous_day_new_cases_per_day',
       'ft08_previous_day_new_deaths_per_day',
       'ft09_previous_day_new_recovered_per_day', 'ft10_cases_evolution_rate',
       'ft11_deaths_evolution_rate', 'ft12_recovered_evolution_rate',
       'ft13_death_percent'],
      dtype='object')

In [4]:
COUNTRY_CODE = 'BR'

target_country = df[(df['countrycode'] == COUNTRY_CODE) & (df['totalcases'].notna()) & (df['ft01_previous_day_totalcases'].notna())]
target_country.head(100)

,date,countrycode,countrylabel,totalcases,totaldeaths,totalrecovered,ft01_previous_day_totalcases,ft02_previous_day_totaldeaths,ft03_previous_day_totalrecovered,ft04_new_cases_per_day,ft05_new_deaths_per_day,ft06_new_recovered_per_day,ft07_previous_day_new_cases_per_day,ft08_previous_day_new_deaths_per_day,ft09_previous_day_new_recovered_per_day,ft10_cases_evolution_rate,ft11_deaths_evolution_rate,ft12_recovered_evolution_rate,ft13_death_percent
394,2020-02-27,BR,Brazil,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1.000000,NaN,NaN,0.000000
395,2020-02-28,BR,Brazil,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,NaN,NaN,0.000000
396,2020-02-29,BR,Brazil,2.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.000000,NaN,NaN,0.000000
397,2020-03-01,BR,Brazil,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.000000,NaN,NaN,0.000000
398,2020-03-02,BR,Brazil,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,NaN,NaN,0.000000
399,2020-03-03,BR,Brazil,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,NaN,NaN,0.000000
400,2020-03-04,BR,Brazil,4.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.000000,NaN,NaN,0.000000
401,2020-03-05,BR,Brazil,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.000000,NaN,NaN,0.000000
402,2020-03-06,BR,Brazil,13.0,0.0,0.0,4.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,3.250000,NaN,NaN,0.000000
403,2020-03-07,BR,Brazil,13.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,1.000000,NaN,NaN,0.000000


In [23]:
# Add new Column to Identify ORIGINAL or COMPUTED Data
target_country.loc[:, ('is_computed')] = [False]

series = target_country.loc[:, ['ft01_previous_day_totalcases', 'totalcases']]

X = []
y = []

for i in range(10, len(series)):

    X.append(
        [
            series.iloc[i]['ft01_previous_day_totalcases'],  # DAY-0
            series.iloc[i-1]['ft01_previous_day_totalcases'],  # DAY-1
            series.iloc[i-2]['ft01_previous_day_totalcases'],   # DAY-2 
            series.iloc[i-3]['ft01_previous_day_totalcases'],   # DAY-3
        ]
    )
    
    y.append(
        series.iloc[i]['totalcases'],  # DAY-0
    )
    
    
    
    
clf = Pipeline(
    [
        ('scaler', preprocessing.StandardScaler()), 
        ('lr', LogisticRegression(random_state=0, solver='liblinear', max_iter=10000))
    ]
)

clf = clf.fit(X, y)

[LibLinear]

c:\projetos\covid-19\venv\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


[[13.0, 13.0, 4.0, 4.0],
 [20.0, 13.0, 13.0, 4.0],
 [25.0, 20.0, 13.0, 13.0],
 [31.0, 25.0, 20.0, 13.0],
 [38.0, 31.0, 25.0, 20.0],
 [52.0, 38.0, 31.0, 25.0],
 [151.0, 52.0, 38.0, 31.0],
 [151.0, 151.0, 52.0, 38.0],
 [162.0, 151.0, 151.0, 52.0],
 [200.0, 162.0, 151.0, 151.0],
 [321.0, 200.0, 162.0, 151.0],
 [372.0, 321.0, 200.0, 162.0],
 [621.0, 372.0, 321.0, 200.0],
 [793.0, 621.0, 372.0, 321.0],
 [1021.0, 793.0, 621.0, 372.0],
 [1593.0, 1021.0, 793.0, 621.0],
 [1593.0, 1593.0, 1021.0, 793.0]]

In [24]:
clf.predict([[1593.0, 1021.0, 793.0, 621.0]])

array([1593.])